<a href="https://colab.research.google.com/github/doronschwartz/NLP/blob/main/HW2/TransferLearningDogsCats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-check-certificate \
   https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O \
   /tmp/cats_and_dogs_filtered.zip


--2024-03-06 15:00:48--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.171.207, 173.194.206.207, 142.251.120.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.171.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   183MB/s    in 0.4s    

2024-03-06 15:00:48 (183 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [2]:
import sys
from matplotlib import pyplot
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator

import os
import zipfile


In [3]:
local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

In [4]:
# Define CNN model with transfer learning
def define_model():
    # Load the VGG16 model with pre-trained weights on ImageNet
    model = VGG16(include_top=False, input_shape=(224, 224, 3))

    # Mark loaded layers as not trainable
    for layer in model.layers:
        layer.trainable = False

    # Add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(128, activation='relu', kernel_initializer='he_uniform')(flat1)
    output = Dense(1, activation='sigmoid')(class1)

    # Define the new model
    model = Model(inputs=model.inputs, outputs=output)

    # Compile the model
    opt = SGD(learning_rate=0.001, momentum=0.9)  # Updated for the warning
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [5]:
# Plot diagnostic learning curves
def summarize_diagnostics(history):
    # Plot loss
    pyplot.subplot(211)
    pyplot.title('Cross Entropy Loss')
    pyplot.plot(history.history['loss'], color='blue', label='train')
    pyplot.plot(history.history['val_loss'], color='orange', label='test')

    # Plot accuracy
    pyplot.subplot(212)
    pyplot.title('Classification Accuracy')
    pyplot.plot(history.history['accuracy'], color='blue', label='train')
    pyplot.plot(history.history['val_accuracy'], color='orange', label='test')

    # Save plot to file
    filename = sys.argv[0].split('/')[-1]
    pyplot.savefig(filename + '_plot.png')
    pyplot.close()

In [6]:
# Run the test harness for evaluating the model
def run_test_harness():
    # Define the model
    model = define_model()

    # Create data generator
    datagen = ImageDataGenerator(featurewise_center=True)

    # Specify ImageNet mean values for centering
    datagen.mean = [123.68, 116.779, 103.939]

    # Prepare iterators
    train_it = datagen.flow_from_directory(train_dir,
                                           class_mode='binary', batch_size=64, target_size=(224, 224))
    test_it = datagen.flow_from_directory(validation_dir,
                                          class_mode='binary', batch_size=64, target_size=(224, 224))

    # Fit the model
    history = model.fit_generator(train_it, steps_per_epoch=len(train_it),
                                  validation_data=test_it, validation_steps=len(test_it), epochs=10, verbose=1)

    # Evaluate the model and test
    _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)
    print('> Accuracy: %.3f' % (acc * 100.0))

    # Plot learning curves
    summarize_diagnostics(history)

In [7]:
# Entry point, run the test harness
run_test_harness()

58889256/58889256 [==============================] - 1s 0us/step
Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


<ipython-input-6-baac1a537cb2>:19: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_it, steps_per_epoch=len(train_it),


Epoch 1/10
32/32 [==============================] - 1839s 58s/step - loss: 1.3890 - accuracy: 0.9100 - val_loss: 0.2130 - val_accuracy: 0.9660
Epoch 2/10
32/32 [==============================] - 1757s 55s/step - loss: 0.0526 - accuracy: 0.9865 - val_loss: 0.1328 - val_accuracy: 0.9700
Epoch 3/10
32/32 [==============================] - 1812s 57s/step - loss: 0.0094 - accuracy: 0.9975 - val_loss: 0.1345 - val_accuracy: 0.9690
Epoch 4/10
32/32 [==============================] - 1765s 56s/step - loss: 0.0037 - accuracy: 0.9995 - val_loss: 0.1388 - val_accuracy: 0.9700
Epoch 5/10
32/32 [==============================] - 1806s 57s/step - loss: 0.0019 - accuracy: 1.0000 - val_loss: 0.1404 - val_accuracy: 0.9680
Epoch 6/10
32/32 [==============================] - 1782s 56s/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 0.1421 - val_accuracy: 0.9690
Epoch 7/10
32/32 [==============================] - 1735s 55s/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.1446 - val_accuracy: 0.9680

<ipython-input-6-baac1a537cb2>:23: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  _, acc = model.evaluate_generator(test_it, steps=len(test_it), verbose=0)


> Accuracy: 96.900
